In [1]:
import pandas as pd
import numpy as np
import regex
from pathlib import Path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
# Excel Data Sets
full_data_2021 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\hrsa\ORG_OTC_FCT_DET.csv')
tx_centers2009 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\txcenters2009clean.csv')

# # Old CSV. A cleaned version is available now.
# tx_centers2009 = pd.read_csv(r'C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\txcenters2009list.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\David Chaniott\\Documents\\Nashville Software School\\capstone\\data\\txcenters2009clean.csv'

In [ ]:
# # This work is finished. New CSV for all of it is cleaned up and included in current code.

# # Delimit everything with commas.
# tx_centers2009['Split'] = tx_centers2009.apply(lambda x: ','.join(x.dropna().values.tolist()), axis=1)
# tx_centers2009 = tx_centers2009['Split'].str.split(',', expand=True)

# # Remove Hanging Spaces
# tx_centers2009 = tx_centers2009.replace(r"^ +| +$", r"", regex=True)
# Remove (ABBR)
tx_centers2009['OPO Name'] = tx_centers2009['OPO Name'].replace(r"\(\w{4}\)", r"", regex=True)

# # This did not work.
# # from pathlib import Path
# # filepath = Path('C:\Users\David Chaniott\Documents\Nashville Software School\capstone\data\test.csv')
# # filepath.parent.mkdir(parents=True, exist_ok=True)  
# # tx_centers2009.to_csv(filepath)

# # This worked
# compression_opts = dict(method='zip',
#                         archive_name='out.csv')  
# tx_centers2009.to_csv('out.zip', index=False,
#           compression=compression_opts)

In [ ]:
# This dataset was created 5-27-2021 by HRSA
# tx_center Count - 254
print(full_data_2021['OTC Name'].nunique())
# State Count - 39
print(full_data_2021.State.nunique())
# Cities Count - 58
print(full_data_2021.City.nunique())

print(full_data_2021.info())

In [ ]:
# This dataset was created 5-2009 by SRTR
# tx_center Count - 327
print(tx_centers2009['OTC Name'].nunique())
# State Count - 48
print(tx_centers2009.State.nunique())
# Cities Count - 169
print(tx_centers2009.City.nunique())

print(tx_centers2009.info())

In [ ]:
# Questions:
# Do surgeons work at different TX Centers?
# Get Footprints of the OPOs

# Match up the 2009 and 2021 lists
# Get the 2021 list noted with NEW hospitals.
# Get the 2009 list noted with CLOSURES
# Get the 2009 list coded with lat.long

In [ ]:
# Merge for the 2009 data.
tx_centers2009
merge_2009 = tx_centers2009

# Make a 2021 list and remove duplicates.
tx_centers2021 = full_data_2021[['OTC Name', 'City', 'State', 'OPO Name', 'OTC Code', 'OPO Provider #']]
tx_centers2021 = tx_centers2021.drop_duplicates()
merge_2021 = tx_centers2021



In [ ]:
# Add Columns ['Year'], ['OTC Code'], ['OPO Provider #']
# Later: address, lat, long region codes

merge_2009['OTC Code'] = 'pending'
merge_2009['OPO Provider #'] = 'pending'
merge_2009['Year'] = 2009
merge_2021['Year'] = 2021
#merge_2009['Year'] = 2009
#merge_2009.loc[:, merge_2009['Year']] = 2009


In [ ]:
tablelist = [merge_2009, merge_2021]
tx_centersall = pd.concat(tablelist)
tx_centersall = tx_centersall.drop_duplicates()

compression_opts = dict(method='zip',
                         archive_name='out.csv')  
tx_centersall.to_csv('out.zip', index=False,
           compression=compression_opts)

In [ ]:
# #tx_centers2021.groupby('OTC Name')['State']
# otc_2021_list = pd.unique(tx_centers2021['OTC Name'])

# otc_2021 = []
# pd.DataFrame(otc_2021)
# otc_2021['OTC Name'] = []
# for otc in otc_2021_list :
#     i = tx_centers2021.loc[tx_centers2021['OTC Name'] == str(otc)]
#     otc_2021['OTC Name'] = i['OTC Name']



In [ ]:
# otc2021 = pd.unique(tx_centers2021['OTC Name'])
# otc2021 = pd.DataFrame(otc2021)
# otc2009 = tx_centers2009
# otc2009 = pd.DataFrame(otc2009)

In [ ]:
# All_TX_dirty
# tx_center Count - 402
print(tx_centersall['OTC Name'].nunique())
# State Count - 48
print(tx_centersall.State.nunique())
# Cities Count - 187
print(tx_centersall.City.nunique())
# OPOs - 114
print(tx_centersall['OPO Name'].nunique())
print(tx_centersall.info())

In [ ]:
# Get OPO Codes and OTC codes from Full Data into tables with unique opos/hopsitals. 
# Then assign join style

opo_2021 = full_data_2021[['OPO Name', 'OPO Provider #']]
otc_2021 = full_data_2021[['OTC Name', 'OTC Code']]

opo_2021 = opo_2021.drop_duplicates()
otc_2021 = otc_2021.drop_duplicates()

opo_codes = dict(zip(opo_2021['OPO Name'], opo_2021['OPO Provider #']))
otc_codes = dict(zip(otc_2021['OTC Name'], otc_2021['OTC Code']))

opo_codes_df = pd.DataFrame.from_dict(opo_codes, orient='index')
otc_codes_df = pd.DataFrame.from_dict(otc_codes, orient='index')

opo_codes_df = opo_codes_df.reset_index()
otc_codes_df = otc_codes_df.reset_index()

In [ ]:
opo_codes_df = opo_codes_df.rename(columns = {'index':'OPO Name', 0:'OPO Provider #'})
otc_codes_df = otc_codes_df.rename(columns = {'index': 'OTC Name', 0: 'OTC Code'})

In [ ]:
# Monday, coming a different way, reset the merge_2009 set because the 2021 set is complete.

merge_2009 = merge_2009.drop(columns=['OPO Provider #', 'OTC Code'])


In [ ]:

#joined = merge_2009.pd.merge(opo_codes_df, how='left', on='OPO Name')
merge_2009['OPO Name'] = merge_2009['OPO Name'].str.strip()


In [ ]:
merge_2009['OTC Name'] = merge_2009['OTC Name'].str.strip()

In [ ]:
merge_2009 = pd.merge(merge_2009, opo_codes_df, how='left', on='OPO Name')

In [ ]:
merge_2009 = pd.merge(merge_2009, otc_codes_df, how='left', on='OTC Name')
merge_2009

In [ ]:
compression_opts = dict(method='zip',
                         archive_name='out.csv')  
merge_2009.to_csv('out.zip', index=False,
           compression=compression_opts)


In [ ]:
compression_opts = dict(method='zip',
                         archive_name='out.csv')  
otc_codes_df.to_csv('out.zip', index=False,
           compression=compression_opts)

In [ ]:
compression_opts = dict(method='zip',
                         archive_name='out.csv')  
opo_codes_df.to_csv('out.zip', index=False,
           compression=compression_opts)